In [1]:
import sys 
sys.path.append('../')
import pandas as pd 
import numpy as np 
import talib
import sqlite3 as sql
import matplotlib.pyplot as plt 
from backtester import * 
from fxcmtoken import my_assets

In [2]:
for i, a in enumerate(my_assets):
    print(i, a)

0 EUR/USD
1 USD/JPY
2 GBP/USD
3 USOil
4 SPX500
5 VOLX
6 XAU/USD
7 Bund
8 BTC/USD


In [21]:
conn = sql.connect('../PriceData/PriceData_m5.db')
ticker = my_assets[8]
print(ticker)
data_sql = pd.read_sql(f"SELECT * FROM '{ticker}'", conn, parse_dates=['date'], index_col=['date'])
for col in ['open', 'close', 'high', 'low']:
    data_sql[col] = data_sql[['bid'+col, 'ask'+col]].mean(axis=1)
conn.close()

BTC/USD


In [22]:
num_periods =  [i for i in range(50, 501, 50)]

for n_period in num_periods:
    data = data_sql.copy(deep=True)
    print(ticker)
    print(f"Period: {n_period:.2f}")

    data['ht'] = talib.HT_TRENDLINE(data.close)
    data['tsf'] = talib.TSF(data.ht, n_period)
    data.dropna(axis=0, inplace=True)
    data['signals'] = np.where(data.ht > data.tsf, 1, -1)
    backtest = IterativeBacktester(data=data,
                                   signals=data.signals,
                                   freq='m5')

    backtest.backtest(progress_bar=False)
    # backtest.results()
    print(f"{backtest.return_df.loc['TotalReturn', 'Portfolio']*100:.4f}%")
    
    print("Signals")
    print(backtest.signals.value_counts())

    print("Signals change")
    print(backtest.signals.diff(1).dropna().apply(np.abs).value_counts())
    
    print("-"*50)

BTC/USD
Period: 50.00
-59.3034%
Signals
-1    5015
 1    4873
Name: signals, dtype: int64
Signals change
0.0    9570
2.0     317
Name: signals, dtype: int64
--------------------------------------------------
BTC/USD
Period: 100.00
-6.1959%
Signals
 1    4953
-1    4885
Name: signals, dtype: int64
Signals change
0.0    9645
2.0     192
Name: signals, dtype: int64
--------------------------------------------------
BTC/USD
Period: 150.00
-8.1836%
Signals
 1    4937
-1    4851
Name: signals, dtype: int64
Signals change
0.0    9644
2.0     143
Name: signals, dtype: int64
--------------------------------------------------
BTC/USD
Period: 200.00
10.7673%
Signals
 1    5156
-1    4582
Name: signals, dtype: int64
Signals change
0.0    9616
2.0     121
Name: signals, dtype: int64
--------------------------------------------------
BTC/USD
Period: 250.00
18.3147%
Signals
 1    5020
-1    4668
Name: signals, dtype: int64
Signals change
0.0    9581
2.0     106
Name: signals, dtype: int64
-----------